In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import datetime
from pprint import pprint as pp

import load_data

/home/molly/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [40]:
def fc_layer(A_prev, size_in, size_out, name="fully-connected"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1))
        b = tf.Variable(tf.constant(0.1, shape=[size_out]))
        act = tf.matmul(A_prev, w) + b
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act, w, b

    
def build_model(x, N_IN, N_HIDDEN):
    parameters = {}
    z1, w1, b1 = fc_layer(x, N_IN, N_HIDDEN, name="fc1")
    parameters.update({"w1": w1, "b1": b1})
    hidden = tf.nn.relu(z1)
    z2, w2, b2 = fc_layer(hidden, N_HIDDEN, N_IN, name="fc2")
    x_recon = z2
    parameters.update({"w2": w2, "b2": b2})
    return x_recon, parameters


def back_prop(x, x_recon, learning_rate):
    loss = tf.reduce_mean(tf.square(x_recon - x))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return train_step


def feed_forward(x, parameters):
    w1, b1 = parameters["w1"], parameters["b1"]
    hidden = tf.nn.relu(tf.matmul(x, w1) + b1)
    w2, b2 = parameters["w2"], parameters["b2"]
    x_recon = tf.matmul(hidden, w2) + b2
    return x_recon, hidden


def mse(x, x_recon, name=""):
    mse = tf.reduce_mean(tf.square(x_recon-x))
    mse_summary = tf.summary.scalar(name + "mse", mse)
    return mse, mse_summary

In [41]:
def train_model(data, batch_size=128, num_epoch=1000, learning_rate=1e-3, extra=""):
    tf.reset_default_graph()
    LOGDIR = "/tmp/tcga_{0}".format(datetime.datetime.today().date())
    N_IN = data.train.num_features
    N_OUT = data.train.num_features
    N_HIDDEN = 1
    
    # train step
    (train_batch, train_iter, val_all, val_iter, 
        train_all, train_iter_all) = data.prep_train_batch(batch_size=batch_size)
    x = train_batch["X"]
    x_recon, parameters = build_model(x, N_IN, N_HIDDEN)
    train_step = back_prop(x, x_recon, learning_rate)
    
    # mse
    x_train, x_val = train_all["X"], val_all["X"]
    x_train_recon, _ = feed_forward(x_train, parameters)
    x_val_recon, _ = feed_forward(x_val, parameters)
    train_mse, train_summ = mse(x_train, x_train_recon, name="train")
    val_mse, val_summ = mse(x_val, x_val_recon, name="valiation")
    
    # run
    sess = tf.Session()
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR + "ae_{0}".format(extra))
    writer.add_graph(sess.graph)                            
    sess.run(tf.global_variables_initializer())

    for epoch in range(num_epoch):
        sess.run([train_iter.initializer])
        t0 = time.time()
        try:
            for i in range(1000):
                if i % 100 == 0:
                    print("step", i)
                    x_step, parameters_step = sess.run([x, parameters])
                    x_step_recon, hidden_step = sess.run(feed_forward(x_step, parameters_step))
                    print("x\n", x_step)
                    print("parameters")
                    pp(parameters_step)
                    print("hideen\n", hidden_step)
                    print("x_recon\n", x_step_recon)
                    mse_step, _ = mse(x_step, x_step_recon)
                    print("mse\n", sess.run(mse_step))
                sess.run(train_step)
        except tf.errors.OutOfRangeError:
            break
            sess.run([train_iter_all.initializer, val_iter.initializer])
#             [train_error, train_s, val_error, val_s] = sess.run(
#                 [train_mse, train_summ, val_mse, val_summ])
#             writer.add_summary(train_s, epoch)
#             writer.add_summary(val_s, epoch)
            
#             if epoch % 10 == 0:
#                 print("epoch", epoch)
#                 print("training mse:", train_error, "validation mse", val_error)
#                 print("epoch time:", time.time()-t0)                             
    sess.close()

In [42]:
tcga = load_data.read_data_sets("./data/mRNA_PCA_0.2_variance_MinMaxScaled.csv")

In [43]:
train_model(tcga, num_epoch=1, batch_size=5)

step 0
x
 [[0.39355198 0.34576556]
 [0.55483913 0.45000488]
 [0.36438873 0.38964662]
 [0.8729454  0.451348  ]
 [0.3172061  0.55868554]]
parameters
{'b1': array([0.1], dtype=float32),
 'b2': array([0.1, 0.1], dtype=float32),
 'w1': array([[-0.05871994],
       [ 0.07276712]], dtype=float32),
 'w2': array([[ 0.07337704, -0.18408872]], dtype=float32)}
hideen
 [[0.10205102]
 [0.10016544]
 [0.10695658]
 [0.08158399]
 [0.12202762]]
x_recon
 [[0.1074882  0.08121356]
 [0.10734984 0.08156067]
 [0.10784816 0.0803105 ]
 [0.10598639 0.08498131]
 [0.10895403 0.07753609]]
mse
 0.16466443
step 100
x
 [[0.5448007  0.21681367]
 [0.32638615 0.43355197]
 [0.46054852 0.33276823]
 [0.49039626 0.3871951 ]
 [0.761899   0.5791727 ]]
parameters
{'b1': array([0.04448926], dtype=float32),
 'b2': array([0.18920435, 0.18915153], dtype=float32),
 'w1': array([[-0.09887452],
       [ 0.01354542]], dtype=float32),
 'w2': array([[ 0.12886654, -0.12449826]], dtype=float32)}
hideen
 [[0.        ]
 [0.01809063]
 [0.00346